# Topic Modeling and Latent Dirichlet Allocation (LDA) in Python

Dataset:<br>
https://www.kaggle.com/therohk/million-headlines/data

In [1]:
import pandas as pd
data = pd.read_csv('dataset/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
print(len(documents))
print(documents[:5])

1103665
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


# Data Pre-processing
* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** — words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** — words are reduced to their root form.

In [3]:
#Loading gensim and nltk libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mk194903/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
stemmer = PorterStemmer()#Stemmers remove morphological affixes from words, leaving only the word stem.
plurals = ['caresses', 'flies', 'dies', 'mules', 'denied', 'died', 'agreed', 'owned', 'humbled', 'sized']
singles = [stemmer.stem(plural) for plural in plurals]
print(' '.join(singles))

caress fli die mule deni die agre own humbl size


In [5]:
# perform lemmatize and stem preprocessing steps on the data set
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [6]:
# Select a document to preview after preprocessing
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [7]:
# Preprocess the headline text, saving the results as ‘processed_docs’
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

### Bag of Words on the Data set 

In [8]:
# Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


###  Gensim filter_extremes
#### Filter out tokens that appear in

* less than 15 documents (absolute number) or
* more than 0.5 documents (fraction of total corpus size, not absolute number).
* after the above two steps, keep only the first 100000 most frequent tokens.

In [9]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

### Gensim doc2bow

For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4021, 1)]

In [11]:
# Preview Bag Of Words for our sample preprocessed document.
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 483 ("rain") appears 1 time.
Word 4021 ("dampen") appears 1 time.


# TF-IDF
Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [12]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903602896750699),
 (1, 0.38524510107363613),
 (2, 0.4974556071174764),
 (3, 0.5055678583740412)]


## LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [13]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [14]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"queensland" + 0.020*"countri" + 0.018*"hospit" + 0.018*"tasmanian" + 0.017*"hour" + 0.015*"work" + 0.015*"children" + 0.013*"student" + 0.013*"citi" + 0.011*"prison"
Topic: 1 
Words: 0.030*"australia" + 0.023*"world" + 0.019*"sydney" + 0.017*"market" + 0.012*"record" + 0.012*"share" + 0.011*"victoria" + 0.009*"port" + 0.009*"melbourn" + 0.009*"australian"
Topic: 2 
Words: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china" + 0.016*"attack" + 0.014*"west" + 0.014*"adelaid" + 0.013*"price" + 0.013*"gold"
Topic: 3 
Words: 0.026*"govern" + 0.016*"plan" + 0.013*"rural" + 0.013*"say" + 0.013*"council" + 0.012*"indigen" + 0.011*"chang" + 0.011*"turnbul" + 0.011*"water" + 0.011*"commun"
Topic: 4 
Words: 0.017*"tasmania" + 0.015*"time" + 0.015*"fight" + 0.013*"lose" + 0.012*"leagu" + 0.011*"john" + 0.010*"life" + 0.009*"premier" + 0.009*"unit" + 0.008*"make"
Topic: 5 
Words: 0.027*"elect" + 0.023*"south" + 0.020*"open" + 0.017*"final" + 0.014*"au

## LDA using TF-IDF

In [15]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.009*"michael" + 0.007*"pacif" + 0.006*"resign" + 0.006*"scott" + 0.004*"corrupt" + 0.004*"australia" + 0.004*"think" + 0.004*"compens" + 0.004*"sunshin" + 0.004*"prime"
Topic: 1 Word: 0.008*"marriag" + 0.008*"david" + 0.007*"juli" + 0.007*"monday" + 0.007*"rugbi" + 0.006*"histori" + 0.006*"andrew" + 0.005*"april" + 0.005*"shorten" + 0.004*"wallabi"
Topic: 2 Word: 0.024*"countri" + 0.022*"hour" + 0.007*"octob" + 0.007*"stori" + 0.006*"wednesday" + 0.005*"zealand" + 0.005*"quiz" + 0.004*"onlin" + 0.004*"tree" + 0.004*"london"
Topic: 3 Word: 0.009*"donald" + 0.009*"australia" + 0.008*"market" + 0.008*"share" + 0.008*"weather" + 0.007*"australian" + 0.007*"world" + 0.006*"ash" + 0.005*"cricket" + 0.005*"test"
Topic: 4 Word: 0.011*"christma" + 0.006*"decemb" + 0.006*"tuesday" + 0.006*"thursday" + 0.005*"island" + 0.004*"solar" + 0.004*"disabl" + 0.004*"paul" + 0.004*"mount" + 0.004*"adelaid"
Topic: 5 Word: 0.016*"rural" + 0.013*"news" + 0.011*"podcast" + 0.008*"live" + 0.00

# Performance evaluation by classifying sample document using LDA Bag of Words model

In [16]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [17]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.41999998688697815	 
Topic: 0.023*"hous" + 0.019*"die" + 0.016*"home" + 0.014*"crash" + 0.014*"break" + 0.013*"perth" + 0.013*"take" + 0.011*"lead" + 0.010*"near" + 0.009*"road"

Score: 0.419996976852417	 
Topic: 0.026*"govern" + 0.016*"plan" + 0.013*"rural" + 0.013*"say" + 0.013*"council" + 0.012*"indigen" + 0.011*"chang" + 0.011*"turnbul" + 0.011*"water" + 0.011*"commun"

Score: 0.02000187709927559	 
Topic: 0.017*"brisban" + 0.016*"famili" + 0.014*"child" + 0.013*"report" + 0.013*"jail" + 0.011*"abus" + 0.011*"sentenc" + 0.011*"show" + 0.010*"victim" + 0.010*"releas"

Score: 0.02000097744166851	 
Topic: 0.034*"polic" + 0.018*"interview" + 0.018*"health" + 0.017*"miss" + 0.014*"investig" + 0.013*"concern" + 0.011*"servic" + 0.011*"search" + 0.011*"call" + 0.011*"minist"

Score: 0.020000169053673744	 
Topic: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china" + 0.016*"attack" + 0.014*"west" + 0.014*"adelaid" + 0.013*"price" + 0.013*"gold"

Score: 0.019

**Note**: Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

# Performance evaluation by classifying sample document using LDA TF-IDF model

In [18]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5643007755279541	 
Topic: 0.010*"govern" + 0.007*"farmer" + 0.007*"turnbul" + 0.007*"water" + 0.006*"farm" + 0.006*"flood" + 0.006*"tasmania" + 0.005*"queensland" + 0.005*"drought" + 0.005*"septemb"

Score: 0.2756865620613098	 
Topic: 0.024*"countri" + 0.022*"hour" + 0.007*"octob" + 0.007*"stori" + 0.006*"wednesday" + 0.005*"zealand" + 0.005*"quiz" + 0.004*"onlin" + 0.004*"tree" + 0.004*"london"

Score: 0.020003488287329674	 
Topic: 0.006*"health" + 0.006*"fund" + 0.006*"budget" + 0.006*"sport" + 0.006*"grandstand" + 0.006*"chang" + 0.005*"coal" + 0.005*"climat" + 0.005*"servic" + 0.005*"indigen"

Score: 0.020002800971269608	 
Topic: 0.017*"polic" + 0.015*"charg" + 0.013*"trump" + 0.013*"murder" + 0.012*"crash" + 0.011*"woman" + 0.009*"death" + 0.009*"court" + 0.008*"jail" + 0.008*"drum"

Score: 0.02000151015818119	 
Topic: 0.016*"rural" + 0.013*"news" + 0.011*"podcast" + 0.008*"live" + 0.007*"abbott" + 0.007*"nation" + 0.007*"friday" + 0.006*"inquest" + 0.005*"anim" + 0.005*

**Note**: Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

# Testing model on unseen document

In [19]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.34999993443489075	 Topic: 0.032*"charg" + 0.030*"court" + 0.023*"murder" + 0.022*"polic" + 0.019*"face"
Score: 0.1833474189043045	 Topic: 0.027*"elect" + 0.023*"south" + 0.020*"open" + 0.017*"final" + 0.014*"australia"
Score: 0.18333333730697632	 Topic: 0.017*"brisban" + 0.016*"famili" + 0.014*"child" + 0.013*"report" + 0.013*"jail"
Score: 0.1833154410123825	 Topic: 0.035*"trump" + 0.022*"kill" + 0.021*"north" + 0.020*"coast" + 0.016*"china"
Score: 0.01667053811252117	 Topic: 0.026*"govern" + 0.016*"plan" + 0.013*"rural" + 0.013*"say" + 0.013*"council"
Score: 0.01666666753590107	 Topic: 0.032*"queensland" + 0.020*"countri" + 0.018*"hospit" + 0.018*"tasmanian" + 0.017*"hour"
Score: 0.01666666753590107	 Topic: 0.030*"australia" + 0.023*"world" + 0.019*"sydney" + 0.017*"market" + 0.012*"record"
Score: 0.01666666753590107	 Topic: 0.017*"tasmania" + 0.015*"time" + 0.015*"fight" + 0.013*"lose" + 0.012*"leagu"
Score: 0.01666666753590107	 Topic: 0.023*"hous" + 0.019*"die" + 0.016*"hom

Ref:<br>
https://gist.github.com/cstrelioff/38b1d16a1253c962b7d7<br>
https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24<br>